In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

# Import data

In [144]:
data_barang = pd.read_csv('../Data/Raw/data_barang.csv', sep=';')
data_rak = pd.read_csv('../Data/Raw/data_rak.csv', sep=';')

In [145]:
data_barang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   No                    209 non-null    float64
 1   Kode Kelompok Barang  209 non-null    float64
 2   Jenis Barang          210 non-null    object 
 3   Kuantitas             209 non-null    object 
 4   Satuan                209 non-null    object 
 5   No. Rak               209 non-null    object 
dtypes: float64(2), object(4)
memory usage: 10.2+ KB


In [146]:
data_barang.dropna(inplace=True)
data_rak.dropna(inplace=True)

In [147]:
def expand_no_rak(row):
    # Split the No. Rak range
    no_rak_range = row['No. Rak'].split(' - ')
    start = int(no_rak_range[0])
    end = int(no_rak_range[1])
    
    # Create a list of dictionaries, each representing a new row
    rows = []
    for rak in range(start, end + 1):
        new_row = row.copy()
        new_row['No. Rak'] = rak
        rows.append(new_row)
    
    return rows

# Apply the function to expand the dataframe
expanded_data = data_barang[data_barang['No. Rak'].str.contains('-')].apply(expand_no_rak, axis=1)

# Flatten the list of lists into a new DataFrame
expanded_data = pd.DataFrame([item for sublist in expanded_data for item in sublist])

In [148]:
data_barang = data_barang[~data_barang['No. Rak'].str.contains('-')]
data_barang = pd.concat([data_barang, expanded_data])

In [149]:
data_rak['No. Rak'] = data_rak['No. Rak'].apply(lambda x: x.split('.')[0])

AttributeError: 'float' object has no attribute 'split'

In [150]:
data_barang['No. Rak'] = data_barang['No. Rak'].astype(int)
data_rak['No. Rak'] = data_rak['No. Rak'].astype(int)

In [151]:
data_barang.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3834 entries, 0 to 171
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   No                    3834 non-null   float64
 1   Kode Kelompok Barang  3834 non-null   float64
 2   Jenis Barang          3834 non-null   object 
 3   Kuantitas             3834 non-null   object 
 4   Satuan                3834 non-null   object 
 5   No. Rak               3834 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 209.7+ KB


In [152]:
data_rak

,Nama Rak,No. Rak,Posisi Rak
0,A,1,A1
1,A,2,A2
2,A,3,A3
3,A,4,A4
4,A,5,A1
...,...,...,...
427,K,428,K4
428,K,429,K1
429,K,430,K2
430,K,431,K3


In [153]:
df = data_barang.merge(data_rak, on='No. Rak', how='inner')

In [154]:
df.drop('No', axis=1, inplace=True)

In [155]:
df['Kuantitas'] = df['Kuantitas'].str.replace(',', '.')
df['Kuantitas'] = pd.to_numeric(df['Kuantitas'])

In [156]:
df.to_csv('../data/processed/df.csv', index=False)

# Modelling

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier

# Feature Engineering

In [158]:
# Encode categorical variables
le = LabelEncoder()
df['Jenis_Barang_econded'] = le.fit_transform(df['Jenis Barang'])
df['Satuan_encoded'] = le.fit_transform(df['Satuan'])
df['Nama_Rak_encoded'] = le.fit_transform(df['Nama Rak'])
df['Posisi_Rak_encoded'] = le.fit_transform(df['Posisi Rak'])

In [159]:
# Select features and targets
features = ['Kode Kelompok Barang', 'Jenis_Barang_econded', 'Kuantitas', 'Satuan_encoded']
targets = ['Posisi_Rak_encoded', 'No. Rak']

X = df[features]
y = df[targets]

In [160]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [161]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [162]:
# KNN Model
knn = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3))
knn.fit(X_train_scaled, y_train)

# Random Forest Model
rf = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
rf.fit(X_train_scaled, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [163]:
# Model Evaluation
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    for i, target in enumerate(targets):
        print(f"\n{model_name} - {target} Accuracy:", accuracy_score(y_test.iloc[:, i], y_pred[:, i]))
        print(f"{model_name} - {target} Classification Report:")
        print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

In [164]:
# Evaluate KNN
evaluate_model(knn, X_test_scaled, y_test, "KNN")


KNN - Posisi_Rak_encoded Accuracy: 0.041278295605858856
KNN - Posisi_Rak_encoded Classification Report:
              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.40      0.50      0.44         4
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         0
           8       0.20      0.33      0.25         3
           9       0.33      0.25      0.29         4
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.29      0.67      0.40         3
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00         1
          15       0.00      0

In [165]:
# Evaluate Random Forest
evaluate_model(rf, X_test_scaled, y_test, "Random Forest")


Random Forest - Posisi_Rak_encoded Accuracy: 0.011984021304926764
Random Forest - Posisi_Rak_encoded Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         4
           5       0.50      0.14      0.22         7
           6       0.17      0.50      0.25         2
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         3
           9       0.50      0.25      0.33         4
          10       0.00      0.00      0.00         1
          11       0.50      1.00      0.67         1
          12       0.33      0.33      0.33         3
          13       0.00      0.00      0.00         4
          14       0.33      1.00      0.50         1
          

In [166]:
def recommend_rack(model, scaler, product_info):
    # Prepare the input
    input_data = np.array([[
        product_info['Kode Kelompok Barang'],
        product_info['Jenis Barang'],
        product_info['Kuantitas'],
        product_info['Satuan_encoded']
    ]])
    
    # Scale the input
    input_scaled = scaler.transform(input_data)
    
    # Make prediction
    predictions = model.predict(input_scaled)[0]
    
    return predictions[0], predictions[1]  # Posisi_Rak_encoded, No. Rak

In [167]:
# Example usage
new_product = {
    'Kode Kelompok Barang': 1000,
    'Jenis Barang': 100203.0,
    'Kuantitas': 5000,
    'Satuan_encoded': 3  # Assuming 'pcs'
}

In [170]:
knn_posisi, knn_no = recommend_rack(knn, scaler, new_product)
rf_posisi, rf_no = recommend_rack(rf, scaler, new_product)

In [171]:
print("\nRecommendation for new product:")
print("KNN Recommendation - Posisi Rak encoded:", knn_posisi, "No. Rak:", knn_no)
print("Random Forest Recommendation - Posisi Rak encoded:", rf_posisi, "No. Rak:", rf_no)


Recommendation for new product:
KNN Recommendation - Posisi Rak encoded: 6 No. Rak: 56
Random Forest Recommendation - Posisi Rak encoded: 10 No. Rak: 68


In [172]:
feature_importance = rf.estimators_[0].feature_importances_
for feature, importance in zip(features, feature_importance):
    print(f"Feature: {feature}, Importance: {importance}")

Feature: Kode Kelompok Barang, Importance: 0.1434748234176526
Feature: Jenis_Barang_econded, Importance: 0.09113886366561741
Feature: Kuantitas, Importance: 0.7125932367673089
Feature: Satuan_encoded, Importance: 0.052793076149421095
